In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [2]:
import sys
sys.path.append('/mnt/Data/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import scipy.signal as scsig
import spectrum as sp
import signale
import cPickle as pkl
import time
import statsmodels.api as sm
import colormaps as cm
import seaborn as sns
import signale.tools as tools
import os

/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/home/chenani/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
def whitenARMA(sig,AR=2,MA=0):
        arma = sm.tsa.ARMA(sig, (AR,MA)).fit(disp=0)
        print 'ARMA parameters calculated for order(%s,%s)' %(AR,MA)
        return arma.resid
def MA(array,ord=2):
    ma = np.array([array[ii-ord/2:ii+ord/2].mean() for ii in range(array.size) if ii >= ord ],dtype=float)
    ma = np.insert(ma,0,array[0:ord/2])
    ma = np.append(ma,array[-(ord/2)-1:-1])
    return ma
def zScore(array):
    return(array - np.average(array))/ np.std(array)

In [5]:
animalPath = '/home/chenani/dataWork/ali/Gerbils/gerbil5/'
localPath = '/mnt/Data/ephysdata/Gerbils/'

In [6]:
lfpPaths = []
for lfp in tools.locate('*.lfp',localPath):
    lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfpPaths = sorted(lfpPaths)
#lfpPaths = [ path for path in lfpPaths if (path.find('leep')>0 and path.find('CSC4')>0) ]
lfpPaths,len(lfpPaths)

(['/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep1_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep2_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep3_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep1_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep2_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep3_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep1_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep2_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep3_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep1_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep2_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep3_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep1_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep2_csc4.lfp',
  '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-1

In [7]:
fig1,ax1 = pl.subplots(7,4,figsize=[20,20],sharex=True,sharey=True)
for iii in range(len(lfpPaths)):
    plotter = ax1[iii/4,np.mod(iii,4)]
    lfp = pkl.load(open(lfpPaths[iii],'rb'))
    Pxx, freqs, t = lfp.spectrogram(minFreq=.1,maxFreq=20,overlap=1024,windowSize=2048,whiten=True)
    BandAvg = []
    plotter.set_title(lfp.tags['path'].split('/')[-1])
    for col in range(Pxx.shape[1]):
        Pxx[:,col] = np.convolve(Pxx[:,col],scsig.gaussian(10,3),'same')
        BandAvg.append(np.array([np.average(Pxx[:,col][0:15]),np.average(Pxx[:,col][16:31]),np.average(Pxx[:,col][32:])]))
    for row in range(Pxx.shape[0]):
        Pxx[row,:] = np.convolve(Pxx[row,:],scsig.gaussian(20,5),'same')
    Pxx = np.clip(Pxx,0,Pxx.mean()+1.4*Pxx.std())
    im = plotter.pcolormesh(t/1000.0,freqs,Pxx,cmap=cm.viridis)
    plotter.axhline(5,c='w',lw=0.5)
    plotter.set_xlim(0,t[0,-1]/1000.0)
    #plotter.plot(lfp.th_del_ratio[0],lfp.th_del_ratio[1],lw=0.5,c='w')
    #plotter.axhline(lfp.th_del_ratio[1].mean()+lfp.th_del_ratio[1].std(),lw=0.5,c='r')
    Draw = np.array(BandAvg)[:,0]
    Traw = np.array(BandAvg)[:,1]
    Graw = np.array(BandAvg)[:,2]
    Dz = (Draw - Draw.mean())/Draw.std()
    Tz = (Traw - Traw.mean())/Traw.std()
    Gz = (Graw - Graw.mean())/Graw.std()
    plotter.plot(t[0,:]/1000.0,zScore(np.convolve(Dz,scsig.gaussian(20,10),'same'))-zScore(np.convolve(Tz,scsig.gaussian(20,10),'same')),'b')
    plotter.axhspan(-1,1,alpha=0.1,color='r')
    plotter.set_ylabel(lfp.tags['file'])
    plotter.set_xlabel(lfp.tags['path'].split('/')[-2])
fig,ax = pl.subplots(3,4,figsize=[20,20],sharex=True,sharey=True)
for iii in range(5):
    plotter = ax[iii/4,np.mod(iii,4)]
    lfp = pkl.load(open(lfpPaths[iii+15],'rb'))
    plotter.plot(lfp.time_axis()/1e3,lfp.signal,lw=0.5)
    BandAvg = []
    plotter.set_xlabel(lfp.tags['path'].split('/')[-2])
    plotter.set_title(lfp.tags['path'].split('/')[-1])


In [7]:
SleepEpochsEven = pd.DataFrame([[1,70,'REM','sleep01','2011-09-19','G6'],
                            [70,135,'SWS','sleep01','2011-09-19','G6'],
                            [140,200,'REM','sleep01','2011-09-19','G6'],
                            [210,275,'SWS','sleep01','2011-09-19','G6'],
                            [275,360,'SWS','sleep01','2011-09-19','G6'],
                            [370,435,'REM','sleep01','2011-09-19','G6'],
                            [435,500,'REM','sleep01','2011-09-19','G6'],
                            [500,565,'REM','sleep01','2011-09-19','G6'],
                            [565,620,'REM','sleep01','2011-09-19','G6'],
                            [630,700,'SWS','sleep01','2011-09-19','G6'],
                            [700,770,'SWS','sleep01','2011-09-19','G6'],
                            [780,850,'REM','sleep01','2011-09-19','G6'],
                            [850,920,'REM','sleep01','2011-09-19','G6'],
                            [920,1000,'REM','sleep01','2011-09-19','G6'],
                            [1000,1060,'SWS','sleep01','2011-09-19','G6'],
                            [1080,1140,'REM','sleep01','2011-09-19','G6'],
                            [1140,1200,'REM','sleep01','2011-09-19','G6'],
                            [20,70,'SWS','sleep02','2011-09-19','G6'],
                            [70,135,'REM','sleep02','2011-09-19','G6'],
                            [135,200,'REM','sleep02','2011-09-19','G6'],
                            [210,270,'SWS','sleep02','2011-09-19','G6'],
                            [280,350,'REM','sleep02','2011-09-19','G6'],
                            [350,420,'REM','sleep02','2011-09-19','G6'],
                            [500,545,'REM','sleep02','2011-09-19','G6'],
                            [545,590,'REM','sleep02','2011-09-19','G6'],
                            [1,65,'SWS','sleep03','2011-09-19','G6'],
                            [65,130,'SWS','sleep03','2011-09-19','G6'],
                            [130,190,'SWS','sleep03','2011-09-19','G6'],
                            [190,255,'SWS','sleep03','2011-09-19','G6'],
                            [260,315,'REM','sleep03','2011-09-19','G6'],
                            [315,370,'REM','sleep03','2011-09-19','G6'],
                            [370,430,'REM','sleep03','2011-09-19','G6'],
                            [435,495,'SWS','sleep03','2011-09-19','G6'],
                            [495,555,'SWS','sleep03','2011-09-19','G6'],
                            [555,615,'SWS','sleep03','2011-09-19','G6'],
                            [615,675,'SWS','sleep03','2011-09-19','G6'],
                            [675,735,'SWS','sleep03','2011-09-19','G6'],
                            [735,795,'SWS','sleep03','2011-09-19','G6'],
                            [795,855,'SWS','sleep03','2011-09-19','G6'],
                            [855,915,'SWS','sleep03','2011-09-19','G6'],
                            [915,975,'SWS','sleep03','2011-09-19','G6'],
                            [975,1030,'SWS','sleep03','2011-09-19','G6'],
                            [1100,1150,'REM','sleep03','2011-09-19','G6'],
                            [1150,1200,'REM','sleep03','2011-09-19','G6'],
                            [115,150,'SWS','sleep01','2011-09-23','G6'],
                            [155,220,'REM','sleep01','2011-09-23','G6'],
                            [220,285,'REM','sleep01','2011-09-23','G6'],
                            [285,360,'REM','sleep01','2011-09-23','G6'],
                            [370,410,'SWS','sleep01','2011-09-23','G6'],
                            [425,500,'SWS','sleep01','2011-09-23','G6'],
                            [500,580,'REM','sleep01','2011-09-23','G6'],
                            [600,650,'SWS','sleep01','2011-09-23','G6'],
                            [660,700,'REM','sleep01','2011-09-23','G6'],
                            [700,760,'SWS','sleep01','2011-09-23','G6'],
                            [760,820,'REM','sleep01','2011-09-23','G6'],
                            [820,880,'REM','sleep01','2011-09-23','G6'],
                            [880,940,'REM','sleep01','2011-09-23','G6'],
                            [950,995,'SWS','sleep01','2011-09-23','G6'],
                            [995,1040,'SWS','sleep01','2011-09-23','G6'],
                            [1050,1110,'REM','sleep01','2011-09-23','G6'],
                            [1110,1165,'REM','sleep01','2011-09-23','G6'],
                            [1165,1220,'REM','sleep01','2011-09-23','G6'],
                            [1220,1240,'SWS','sleep01','2011-09-23','G6'],
                            [1240,1320,'REM','sleep01','2011-09-23','G6'],
                            [40,80,'SWS','sleep02','2011-09-23','G6'],
                            [85,140,'REM','sleep02','2011-09-23','G6'],
                            [140,195,'REM','sleep02','2011-09-23','G6'],
                            [195,250,'REM','sleep02','2011-09-23','G6'],
                            [250,310,'REM','sleep02','2011-09-23','G6'],
                            [310,370,'REM','sleep02','2011-09-23','G6'],
                            [370,430,'REM','sleep02','2011-09-23','G6'],
                            [430,490,'REM','sleep02','2011-09-23','G6'],
                            [490,570,'SWS','sleep02','2011-09-23','G6'],
                            [610,670,'SWS','sleep02','2011-09-23','G6'],
                            [670,730,'SWS','sleep02','2011-09-23','G6'],
                            [730,800,'REM','sleep02','2011-09-23','G6'],
                            [800,880,'REM','sleep02','2011-09-23','G6'],
                            [1,70,'REM','sleep03','2011-09-23','G6'],
                            [70,110,'SWS','sleep03','2011-09-23','G6'],
                            [110,135,'REM','sleep03','2011-09-23','G6'],
                            [140,200,'SWS','sleep03','2011-09-23','G6'],
                            [300,350,'REM','sleep03','2011-09-23','G6'],
                            [360,405,'SWS','sleep03','2011-09-23','G6'],
                            [405,450,'SWS','sleep03','2011-09-23','G6'],
                            [550,660,'SWS','sleep03','2011-09-23','G6'],
                            [550,605,'SWS','sleep03','2011-09-23','G6'],
                            [605,660,'SWS','sleep03','2011-09-23','G6'],
                            [680,735,'REM','sleep03','2011-09-23','G6'],
                            [735,790,'REM','sleep03','2011-09-23','G6'],
                            [790,845,'REM','sleep03','2011-09-23','G6'],
                            [845,900,'REM','sleep03','2011-09-23','G6'],
                            [930,1000,'SWS','sleep03','2011-09-23','G6'],
                            [1000,1065,'SWS','sleep03','2011-09-23','G6'],
                            [1065,1130,'SWS','sleep03','2011-09-23','G6'],
                            [1,55,'SWS','sleep01','2011-08-05','G5'],
                            [55,110,'SWS','sleep01','2011-08-05','G5'],
                            [110,165,'REM','sleep01','2011-08-05','G5'],
                            [165,220,'REM','sleep01','2011-08-05','G5'],
                            [220,280,'REM','sleep01','2011-08-05','G5'],
                            [280,340,'REM','sleep01','2011-08-05','G5'],
                            [340,400,'REM','sleep01','2011-08-05','G5'],
                            [400,475,'SWS','sleep01','2011-08-05','G5'],
                            [475,550,'SWS','sleep01','2011-08-05','G5'],
                            [1,55,'SWS','sleep02','2011-08-05','G5'],
                            [55,110,'SWS','sleep02','2011-08-05','G5'],
                            [170,220,'REM','sleep02','2011-08-05','G5'],
                            [120,195,'REM','sleep02','2011-08-05','G5'],
                            [195,270,'REM','sleep02','2011-08-05','G5'],
                            [420,500,'SWS','sleep02','2011-08-05','G5'],
                            [520,550,'REM','sleep02','2011-08-05','G5'],
                            [550,600,'SWS','sleep02','2011-08-05','G5'],
                            [1,75,'SWS','sleep03','2011-08-05','G5'],
                            [100,190,'SWS','sleep03','2011-08-05','G5'],
                            [210,260,'SWS','sleep03','2011-08-05','G5'],
                            [260,310,'SWS','sleep03','2011-08-05','G5'],
                            [310,360,'REM','sleep03','2011-08-05','G5'],
                            [535,590,'SWS','sleep03','2011-08-05','G5'],
                            [1,35,'REM','sleep01','2011-08-07','G5'],
                            [40,80,'SWS','sleep01','2011-08-07','G5'],
                            [80,135,'REM','sleep01','2011-08-07','G5'],
                            [135,200,'SWS','sleep01','2011-08-07','G5'],
                            [200,265,'SWS','sleep01','2011-08-07','G5'],
                            [265,350,'SWS','sleep01','2011-08-07','G5'],
                            [370,460,'REM','sleep01','2011-08-07','G5'],
                            [5,85,'SWS','sleep02','2011-08-07','G5'],
                            [90,160,'REM','sleep02','2011-08-07','G5'],
                            [160,230,'REM','sleep02','2011-08-07','G5'],
                            [230,300,'REM','sleep02','2011-08-07','G5'],
                            [300,370,'REM','sleep02','2011-08-07','G5'],
                            [380,445,'SWS','sleep02','2011-08-07','G5'],
                            [445,510,'SWS','sleep02','2011-08-07','G5'],
                            [510,575,'SWS','sleep02','2011-08-07','G5'],
                            [575,640,'SWS','sleep02','2011-08-07','G5'],
                            [640,710,'SWS','sleep02','2011-08-07','G5'],
                            [1,100,'SWS','sleep03','2011-08-07','G5'],
                            [120,170,'REM','sleep03','2011-08-07','G5'],
                            [180,240,'SWS','sleep03','2011-08-07','G5'],
                            [240,300,'SWS','sleep03','2011-08-07','G5'],
                            [300,360,'SWS','sleep03','2011-08-07','G5'],
                            [360,420,'SWS','sleep03','2011-08-07','G5'],
                            [420,480,'SWS','sleep03','2011-08-07','G5'],
                            [480,530,'SWS','sleep03','2011-08-07','G5'],
                            [530,580,'SWS','sleep03','2011-08-07','G5'],
                            [1,50,'REM','sleep01','2011-08-09','G5'],
                            [50,140,'SWS','sleep01','2011-08-09','G5'],
                            [140,220,'REM','sleep01','2011-08-09','G5'],
                            [220,265,'SWS','sleep01','2011-08-09','G5'],
                            [265,335,'REM','sleep01','2011-08-09','G5'],
                            [335,385,'SWS','sleep01','2011-08-09','G5'],
                            [390,470,'REM','sleep01','2011-08-09','G5'],
                            [475,530,'SWS','sleep01','2011-08-09','G5'],
                            [530,585,'SWS','sleep01','2011-08-09','G5'],
                            [585,640,'SWS','sleep01','2011-08-09','G5'],
                            [640,700,'SWS','sleep01','2011-08-09','G5'],
                            [1,50,'REM','sleep01','2011-08-09','G5'],
                            [20,90,'SWS','sleep02','2011-08-09','G5'],
                            [150,200,'SWS','sleep02','2011-08-09','G5'],
                            [200,250,'SWS','sleep02','2011-08-09','G5'],
                            [260,320,'REM','sleep02','2011-08-09','G5'],
                            [320,380,'REM','sleep02','2011-08-09','G5'],
                            [380,440,'REM','sleep02','2011-08-09','G5'],
                            [440,500,'REM','sleep02','2011-08-09','G5'],
                            [1,50,'REM','sleep03','2011-08-09','G5'],
                            [50,110,'SWS','sleep03','2011-08-09','G5'],
                            [110,170,'SWS','sleep03','2011-08-09','G5'],
                            [170,230,'SWS','sleep03','2011-08-09','G5'],
                            [230,300,'SWS','sleep03','2011-08-09','G5'],
                            [320,370,'REM','sleep03','2011-08-09','G5'],
                            [370,420,'REM','sleep03','2011-08-09','G5'],
                            [420,490,'SWS','sleep03','2011-08-09','G5'],
                            [490,560,'SWS','sleep03','2011-08-09','G5'],
                            [560,600,'REM','sleep03','2011-08-09','G5'],
                            [620,690,'SWS','sleep03','2011-08-09','G5'],
                            [10,65,'SWS','sleep01','2011-08-10','G5'],
                            [65,120,'SWS','sleep01','2011-08-10','G5'],
                            [120,160,'REM','sleep01','2011-08-10','G5'],
                            [160,240,'SWS','sleep01','2011-08-10','G5'],
                            [250,300,'REM','sleep01','2011-08-10','G5'],
                            [300,350,'SWS','sleep01','2011-08-10','G5'],
                            [355,390,'REM','sleep01','2011-08-10','G5'],
                            [440,520,'SWS','sleep01','2011-08-10','G5'],
                            [520,560,'REM','sleep01','2011-08-10','G5'],
                            [570,595,'SWS','sleep01','2011-08-10','G5'],
                            [80,140,'SWS','sleep02','2011-08-10','G5'],
                            [140,200,'SWS','sleep02','2011-08-10','G5'],
                            [200,285,'SWS','sleep02','2011-08-10','G5'],
                            [380,450,'REM','sleep02','2011-08-10','G5'],
                            [450,520,'REM','sleep02','2011-08-10','G5'],
                            [10,65,'REM','sleep03','2011-08-10','G5'],
                            [65,120,'REM','sleep03','2011-08-10','G5'],
                            [120,195,'SWS','sleep03','2011-08-10','G5'],
                            [200,275,'REM','sleep03','2011-08-10','G5'],
                            [275,350,'REM','sleep03','2011-08-10','G5'],
                            [360,410,'SWS','sleep03','2011-08-10','G5'],
                            [410,460,'SWS','sleep03','2011-08-10','G5'],
                            [520,590,'SWS','sleep03','2011-08-10','G5'],
                            [620,680,'SWS','sleep03','2011-08-10','G5'],
                            ],
                           columns=['t0','t1','epoch','session','date','animal'])


In [15]:
psd = np.array([])
dates = []
sessions = []
animals = []
state = []
freqz = []
epN = 0
epochNo = []
for path in lfpPaths[::-4]:
    pickedChannel = pkl.load(open(path,'rb'))
    NFFT = int(2 ** 17)
    expDate = path.split('_')[0].split('/')[-1]
    session = path.split('_')[-2]
    session = session[:-1]+'0'+session[-1]
    relevantEpochs = SleepEpochsEven[np.logical_and(SleepEpochsEven['date']== expDate,
                                                SleepEpochsEven['session']==session)]
    relevantEpochs.index = range(relevantEpochs.index.size)
    Nepochs = relevantEpochs.shape[0]
    print expDate,session
    if Nepochs:
        print NepochsresidPSDSmooth
        print relevantEpochs
        for kk in range(Nepochs):
            t0 = relevantEpochs.t0[kk]
            tf = relevantEpochs.t1[kk]
            sigSlice = np.logical_and(pickedChannel.time_axis()>t0*1e3,pickedChannel.time_axis()<tf*1e3)
            dsig = scsig.decimate(pickedChannel.signal[sigSlice],2)
            arma = sm.tsa.ARMA(dsig,(2,0)).fit(disp=0)
            #residPSD = sp.Periodogram(arma.resid,sampling=1000,NFFT=NFFT)
            residPSD = sp.Periodogram(dsig,sampling=1000,NFFT=NFFT)
            residPSD()
            Nfreqz = residPSD.psd.size
            smoothingKernel = scsig.gaussian(Nfreqz/100,Nfreqz/200)
            residPSDSmooth = np.convolve(residPSD.psd*np.power(residPSD.frequencies(),2),smoothingKernel,'same')
            residPSDSmooth = scsig.decimate(residPSDSmooth,10)
            residPSDSmooth = 10*np.log10(residPSDSmooth/residPSDSmooth[0])
            psd = np.append(psd,residPSDSmooth)
            freqz.append(scsig.decimate(np.array(residPSD.frequencies()),10))
            dates.append(residPSDSmooth.size*[relevantEpochs.date[0]])
            sessions.append(residPSDSmooth.size*[relevantEpochs.session[0]])
            animals.append(residPSDSmooth.size*[relevantEpochs.animal[0]])
            state.append(residPSDSmooth.size*[relevantEpochs.epoch[kk]])
            epochNo.append(residPSDSmooth.size*[epN])
            epN+=1
animals = np.array(animals).flatten()
dates = np.array(dates).flatten()
sessions = np.array(sessions).flatten()
state = np.array(state).flatten()
freqz = np.array(freqz).flatten()
epochNo = np.array(epochNo).flatten()
dfm = np.array([psd,freqz,state,epochNo,sessions,dates,animals]).T
df = pd.DataFrame(dfm,columns=['psd','f','epoch','epochNo','session','date','animal'])
df[['psd', 'f']] = df[['psd', 'f']].astype(float)
df[['epochNo']] = df[['epochNo']].astype(int)
df[['epoch','session','date','animal']] = df[['epoch','session','date','animal']].astype(str)

2011-09-23 sleep03
17
      t0    t1 epoch  session        date animal
0      1    70   REM  sleep03  2011-09-23     G6
1     70   110   SWS  sleep03  2011-09-23     G6
2    110   135   REM  sleep03  2011-09-23     G6
3    140   200   SWS  sleep03  2011-09-23     G6
4    300   350   REM  sleep03  2011-09-23     G6
5    360   405   SWS  sleep03  2011-09-23     G6
6    405   450   SWS  sleep03  2011-09-23     G6
7    550   660   SWS  sleep03  2011-09-23     G6
8    550   605   SWS  sleep03  2011-09-23     G6
9    605   660   SWS  sleep03  2011-09-23     G6
10   680   735   REM  sleep03  2011-09-23     G6
11   735   790   REM  sleep03  2011-09-23     G6
12   790   845   REM  sleep03  2011-09-23     G6
13   845   900   REM  sleep03  2011-09-23     G6
14   930  1000   SWS  sleep03  2011-09-23     G6
15  1000  1065   SWS  sleep03  2011-09-23     G6
16  1065  1130   SWS  sleep03  2011-09-23     G6
2011-09-19 sleep02
8
    t0   t1 epoch  session        date animal
0   20   70   SWS  sleep02  2

In [11]:
freqBins = np.linspace(0.5,300.5,600)
psd = np.array([])
dates = []
sessions = []
animals = []
state = []
freqz = []
epN = 0
epochNo = []
for path in lfpPaths[:]:
    pickedChannel = pkl.load(open(path,'rb'))
    expDate = path.split('_')[0].split('/')[-1]
    session = path.split('_')[-2]
    session = session[:-1]+'0'+session[-1]
    relevantEpochs = SleepEpochs[np.logical_and(SleepEpochs['date']== expDate,
                                                SleepEpochs['session']==session)]
    relevantEpochs.index = range(relevantEpochs.index.size)
    Nepochs = relevantEpochs.shape[0]
    print expDate,session
    if Nepochs:
        #print Nepochs
        #print relevantEpochs
        for kk in range(Nepochs):
            t0 = relevantEpochs.t0[kk]
            tf = relevantEpochs.t1[kk]
            sigSlice = np.logical_and(pickedChannel.time_axis()>t0*1e3,pickedChannel.time_axis()<tf*1e3)
            #missing sth
            epN+=1
animals = np.array(animals).flatten()
dates = np.array(dates).flatten()
sessions = np.array(sessions).flatten()
state = np.array(state).flatten()
freqz = np.array(freqz).flatten()
epochNo = np.array(epochNo).flatten()
dfm = np.array([psd,freqz,state,epochNo,sessions,dates,animals]).T
df = pd.DataFrame(dfm,columns=['psd','f','epoch','epochNo','session','date','animal'])
df[['psd', 'f']] = df[['psd', 'f']].astype(float)
df[['epochNo']] = df[['epochNo']].astype(int)
df[['epoch','session','date','animal']] = df[['epoch','session','date','animal']].astype(str)

SyntaxError: invalid syntax (<ipython-input-11-0125245ba182>, line 27)

In [17]:
#pl.yscale('log')
sns.tsplot(value='psd',time='f',condition='epoch',unit='epochNo',data=df)#[df['epoch']=='SWS'])
pl.xlim(0,450)

(0, 450)

In [18]:
df.to_pickle(localPath+'psdF2dB.pd')
#SleepEpochsEven.to_pickle(localPath+'SleepEpochsEven.pd')

In [32]:
(SleepEpochsEven[SleepEpochsEven['epoch']=='REM']['t1']-SleepEpochsEven[SleepEpochsEven['epoch']=='REM']['t0']).hist(bins=40)
(SleepEpochsEven[SleepEpochsEven['epoch']=='SWS']['t1']-SleepEpochsEven[SleepEpochsEven['epoch']=='SWS']['t0']).hist(bins=40,alpha=0.5)
pl.show()

In [25]:
SleepEpochsEven[(SleepEpochsEven.t1 - SleepEpochsEven.t0) >100]

,t0,t1,epoch,session,date,animal
84,550,660,SWS,sleep03,2011-09-23,G6
92,980,1130,SWS,sleep03,2011-09-23,G6
